In [1]:
import os 
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import requests
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
#LOading the environment and checking if this api key exist or not 
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
# checking if the model is working
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to have you here. How can I assist you today?


In [5]:
# some websites need to use proper header for fetching the data 
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
#A system prompt will tell them what task they are performing and what tone they should use
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [14]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

The **Anthropic** website primarily focuses on their AI model, **Claude**, and its capabilities. The site features several key areas:

## AI Models
- Introduction to **Claude Opus 4** and **Sonnet 4**, the latest and most advanced AI models.
- Aimed at enhancing productivity in coding and AI agent development.

## Features and Services
- **API** for developers to create custom applications utilizing Claude.
- Information on various pricing plans including Max, Team, Enterprise, and Education options.

## Company Values
- Commitment to **AI safety and ethical development** with a focus on serving humanity's long-term well-being.
- Transparency in their processes and responsible scaling policy.

## Research and Insights
- Ongoing research into AI's societal impacts, with publications highlighting findings (e.g., alignment faking in large language models).

## News and Announcements
- Recent announcements include the release of **ISO 42001 certification**.
- Continuous updates on new features and model improvements.

## Learning Resources
- Access to **Anthropic Academy** for educational resources and customer stories to help users build effectively with Claude.

Overall, the website positions Anthropic as a leader in responsible AI, combining advanced technology with a commitment to safety and ethical standards.